In [ ]:
import datetime as dt
import io
import json
import pandas as pd
import requests as rq
import zipfile as zf

from chardet import detect
from sqlalchemy import create_engine

In [ ]:
HIST_UID = "dczd2t"
api_calls = 0

In [ ]:
frames = dict()

with open("../assets/datasets.json") as _datasets_:
    sets = json.load(_datasets_)
    hists = \
        [
            _ for _ in sets["results"]
            if _["dataset_uid"] == f"da_{HIST_UID}"
        ]

    if len(hists) == 1:
        hists = hists[0]

        for hist in hists["attachments"]:
            request = rq.get(hist["url"])

            with zf.ZipFile(io.BytesIO(request.content)) as _zip_:
                filename = hist["title"].split(".zip")[0]
                title = filename.split(".csv")[0]
                encoding = "UTF-8"

                with _zip_.open(filename) as file:
                    data = file.read()
                    encoding = detect(data[:10000])
                
                frames[title] = \
                    pd.read_csv(
                        _zip_.open(filename),
                        sep=";",
                        encoding=encoding["encoding"]
                    )

In [ ]:
dfs = frames.copy()

new_columns = {
    "OBJECTID": "object_id",
    "OID_": "object_id",
    "ID_DMR": "id_dmr",
    "TYPE": "type",
    "SOUSTYPE": "sous_type",
    "soustype": "sous_type",
    "ADRESSE": "adresse",
    "CODE_POSTAL": "code_postal",
    "VILLE": "ville",
    "ARRONDISSEMENT": "arrondissement",
    "DATEDECL": "date_collecte",
    "datedecl": "date_collecte",
    "ANNEEDECL": "annee_collecte",
    "anneedecl": "annee_collecte",
    "MOISDECL": "mois_collecte",
    "moisdecl": "mois_collecte",
    "ETAT": "etat",
    "DATEETAT": "date_etat",
    "dateetat": "date_etat",
    "NUMERO": "numero",
    "PREFIXE": "prefixe",
    "INTERVENANT": "intervenant",
    "CONSEILQUARTIER": "conseil_quartier",
    "conseilquartier": "conseil_quartier",
    "X": "longitude",
    "Y": "latitude",
}

for key in dfs.keys():
    dfs[key] = dfs[key].rename(columns=new_columns)

    if "object_id" not in dfs[key].columns:
        dfs[key]["object_id"] = 0

    if "etat" not in dfs[key].columns:
        dfs[key]["etat"] = "Etat inconnnu"

    if "date_etat" not in dfs[key].columns:
        next_year = int(key.split("_")[1])
        dfs[key]["date_etat"] = \
            dt.datetime(next_year, 1, 1).strftime("%d/%m/%Y %H:%M:%S")

In [ ]:
df_full = pd.concat(list(dfs.values()), ignore_index=True)

In [ ]:
df_full["ville"] = df_full["ville"].apply(
    lambda v: \
        f"Paris {v.split("aris 0")[1]}" \
            if "aris 0" in v \
            else v
)

In [ ]:
df_full["arrondissement"] = df_full["arrondissement"].apply(
    lambda a: \
        str(a).split(",")[0] \
            if len(str(a).split(",")[0]) == 2 \
            else f"0{str(a).split(",")[0]}"
)

In [ ]:
df_full["code_postal"] = \
    df_full["code_postal"] \
        .fillna(df_full["arrondissement"]) \
        .astype("string") \
        .apply(lambda c: \
            f"750{c}" if len(c) == 2 \
                else c.split('.')[0] if '.'in c \
                else c.split(',')[0] if ','in c \
                else c
        )

In [ ]:
df_full["latitude"] = \
    df_full["latitude"].apply(
        lambda lat: float(lat.replace(",", "."))
    ).astype("float")

df_full["longitude"] = \
    df_full["longitude"].apply(
        lambda lon: float(lon.replace(",", "."))
    ).astype("float")

In [ ]:
df_full["coords"] = \
    (df_full["longitude"].astype("string") + "," + df_full["latitude"].astype("string")) \
    .apply(
        lambda c: (float(c.split(",")[0]), float(c.split(",")[1]))
    )

In [ ]:
if df_full["adresse"].isna().sum() > 0:
    missing = {}
    indexes = \
        df_full[df_full["adresse"].isna()].index.to_list()
    
    for _ in indexes:
        missing[str(_)] = (
            df_full.loc[_, "latitude"],
            df_full.loc[_, "longitude"]
        )

    for _ in missing.keys():
        values = [
            a for a \
                in df_full[df_full["coords"] == missing[_]]["adresse"].unique() \
                if type(a) == str
        ]
    
        if len(values) > 0:
            df_full.at[int(_), "adresse"] = values[0]

In [ ]:
if df_full["conseil_quartier"].isna().sum() > 0:
    indexes = \
        df_full[df_full["conseil_quartier"].isna()].index.to_list()
    
    for _ in indexes:
        missing[str(_)] = df_full.loc[_, "adresse"]

    for _ in missing.keys():
        values = [
            a for a \
                in df_full[df_full["adresse"] == missing[_]]["conseil_quartier"].unique() \
                if type(a) == str
        ]
    
        if len(values) > 0:
            df_full.at[int(_), "conseil_quartier"] = values[0]

In [ ]:
df_full["sous_type"] = df_full["sous_type"].fillna("Sous-type inconnu")
df_full["intervenant"] = df_full["intervenant"].fillna("Intervenant inconnu")
df_full["numero"] = df_full["numero"].apply(lambda n: int(str(n).split(",")[0]))

In [ ]:
print(df_full.shape)
print(df_full.isna().sum())
df_full = df_full.dropna()
print(df_full.shape)

In [ ]:
#df_full.to_csv("./DMR_history.csv", sep=";")

#df_full.to_parquet("./DMR_history.parquet")

#engine = create_engine("sqlite:///./dmr_hist.db")
#df_full.to_sql("./DMR_history.sql", engine)